# Higgins - Time-Dependent Eulerian Current - Data Generator

##### Includes: Seasonal averaging, global viscosity model, deep water linear dispersion

## 1. Setting Environment

In [1]:
import xarray
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as mticker
import numpy as np
import math
from datetime import date, timedelta
import ftplib
import xarray
from tqdm.autonotebook import tqdm
import glob
import cmath
import os
from mpl_toolkits.axes_grid1.inset_locator import InsetPosition
import warnings
from scipy import special
import time
import multiprocessing as mp
import itertools
import concurrent.futures
from matplotlib.collections import LineCollection
from datetime import date, datetime, timedelta
warnings.filterwarnings("ignore", category=RuntimeWarning) 

/Users/jakecunningham/opt/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


## 2. Data Preparation

In [2]:
def loader(path):
    paths = glob.glob(path)
    paths.sort(key=os.path.getmtime)
    file = xarray.open_mfdataset(paths, combine='by_coords')
    return file

In [3]:
def loader_Stokes(path):
    file = xarray.open_mfdataset(path, combine='by_coords')
    return file

In [4]:
def dates_maker(start,end):
    dates = []
    current = start
    dates.append(current.strftime('%Y%m%d'))
    while current < end:
        current += timedelta(days=1)
        dates.append(current.strftime('%Y%m%d'))
    return dates

In [5]:
def higgins_convolution_parallel_variable_f_variable_k_equator1(lat):
    
    kernel_length = 300

    global lons
    
    Ue_point = np.zeros([len(Us_u_array[:,0,0]),1,720]) + 0j
    print(lat)
            
    kernel = np.zeros([300,1]) + 0j

    w = 7.2921 * (10**-5) * 3600
    latitude = np.linspace(-78,80,317)[lat]
    
    for lon in tqdm(range(lons)):
        latitude_rad = latitude * np.pi/180
        f = 2*w*np.sin(latitude_rad)
        v = Az_array[lat,lon] * 3600
        k = k_array[lat,lon]

        for n in range(kernel_length):
            if n > 0:
                t = 3*n
                K = (((2*k*(v**0.5))/((t*np.pi)**0.5)))*np.exp(-1j*f*t) - ((1j*f*np.exp(-1j*f*t)) *special.erfcx((4*k*k*v*t)**0.5))
                kernel[n-1] = K
    
        U_forcing_list = Us_u_array[:,lat,lon] + (1j * Us_v_array[:,lat,lon])
    
        if math.isnan(U_forcing_list[0]) == 0:
            U_forcing = np.reshape(U_forcing_list, (1,len(U_forcing_list)))
    
            for i in range(len(U_forcing_list)-1):
                j = i+1
                if j <= kernel_length:
                    Us = U_forcing[0][:j]
                    K = kernel[:j]
                    Us_reverse = np.flip(Us)
                    Us_vector = np.reshape(Us_reverse, (1,j))
                else:
                    Us = U_forcing[0][j-kernel_length:j]
                    K = kernel[:kernel_length]
                    Us_reverse = np.flip(Us)
                    Us_vector = np.reshape(Us_reverse, (1,kernel_length))
                matrix = np.dot(K,Us_vector)
                Ue = sum(np.diag(matrix))
                Ue_point[i+1,0,lon]=Ue*3
                
    del kernel
        
    return Ue_point

In [6]:
start = datetime(2002,1,1)
end = datetime(2014,12,31)
delta = timedelta(days=1)
dates = []
current = start
dates.append(current.strftime('%Y%m%d'))
while current < end:
    current += delta
    dates.append(current.strftime('%Y%m%d'))

In [7]:
years = np.linspace(2002,2014,13)

start = []
end = []

start.append(datetime(2002,1,1).strftime('%Y%m%d'))
end.append(datetime(2002,2,28).strftime('%Y%m%d'))

for year in years:
    year = int(year)
    spring_start = datetime(year,3,1)
    spring_end = datetime(year,5,31)
    start.append(spring_start.strftime('%Y%m%d'))
    end.append(spring_end.strftime('%Y%m%d'))
    
    summer_start = datetime(year,6,1)
    summer_end = datetime(year,8,31)
    start.append(summer_start.strftime('%Y%m%d'))
    end.append(summer_end.strftime('%Y%m%d'))
    
    autumn_start = datetime(year,9,1)
    autumn_end = datetime(year,11,30)
    start.append(autumn_start.strftime('%Y%m%d'))
    end.append(autumn_end.strftime('%Y%m%d'))
    
    winter_start = datetime(year,12,1)
    winter_end = datetime(year+1,2,28)
    start.append(winter_start.strftime('%Y%m%d'))
    end.append(winter_end.strftime('%Y%m%d'))
    
end[end.index('20040228')] = '20040229'
end[end.index('20080228')] = '20080229'
end[end.index('20120228')] = '20120229'

index_start = []
for d in start:
    index_start.append(dates.index(d)*8)
index_start.append(37984)

len(index_start)

54

In [8]:
print(index_start)

[0, 472, 1208, 1944, 2672, 3392, 4128, 4864, 5592, 6320, 7056, 7792, 8520, 9240, 9976, 10712, 11440, 12160, 12896, 13632, 14360, 15080, 15816, 16552, 17280, 18008, 18744, 19480, 20208, 20928, 21664, 22400, 23128, 23848, 24584, 25320, 26048, 26768, 27504, 28240, 28968, 29696, 30432, 31168, 31896, 32616, 33352, 34088, 34816, 35536, 36272, 37008, 37736, 37984]


In [12]:
for master in range(len(index_start)-1):
    master = master + 52 
    j = index_start[master+1]
    i = index_start[master] - 300
    m = index_start[master]
    
    if i < 0:
        i = 0
        
    stamp = j
    print(stamp)
    
    location_stokes = '/Volumes/4YP/Data/Stokes_3hr/Stokes'
    file_type = '.nc'
    paths_stokes = []
    for n in range(i,j):
        s = str(n).zfill(6)
        paths_stokes.append(location_stokes + s + file_type)
    Us_file = loader_Stokes(paths_stokes)
    Us_u_array = Us_file.uuss.values
    Us_v_array = Us_file.vuss.values
    time = Us_file.uuss['time'].values
    
    location_PFreq = '/Volumes/4YP/Data/PFreq_3hr/PFreq'
    file_type = '.nc'
    paths_PFreq = []
    for n in range(m,j):
        s = str(n).zfill(6)
        paths_PFreq.append(location_PFreq + s + file_type)
    PF_file = loader_Stokes(paths_PFreq)
    k_file = ((2*np.pi*PF_file.fp)**2)/9.81
    k_array = k_file.mean('time').values
    
    location_viscosity = '/Volumes/JCHD/Microplastics_Paper/Data/Az_24hr/Az'
    file_type = '.nc'
    paths_viscosity = []
    ds = dates[int(m/8):int(j/8)]
    if master > 38:
        new_master = master - 4
        j = index_start[new_master+1]
        i = index_start[new_master] - 300
        m = index_start[new_master]
    if master > 42:
        new_master = master - 8
        j = index_start[new_master+1]
        i = index_start[new_master] - 300
        m = index_start[new_master]
    if master > 46:
        new_master = master - 12
        j = index_start[new_master+1]
        i = index_start[new_master] - 300
        m = index_start[new_master]
    if master > 50:
        new_master = master - 16
        j = index_start[new_master+1]
        i = index_start[new_master] - 300
        m = index_start[new_master]
    ds = dates[int(m/8):int(j/8)]
    for i in range(len(ds)):
        paths_viscosity.append(location_viscosity + ds[i] + file_type)
    Az_file = loader_Stokes(paths_viscosity)
    Az_array = Az_file.Az.mean('time').values
    
    lat = list(range(len(Us_file.latitude)))
    lons = len(Us_file.longitude)

    cpus = mp.cpu_count()
    p=mp.Pool(cpus)
    output = p.map(higgins_convolution_parallel_variable_f_variable_k_equator1, tqdm(lat))
    data = np.concatenate(output, axis=1)

    dims = ('time', 'latitude', 'longitude')

    U_ek = xarray.Dataset(
        data_vars={
                'U': (dims, data.real),
                'V': (dims, data.imag)},
        coords={
                'time': Us_file.time,
                'latitude': Us_file.latitude,
                'longitude': Us_file.longitude}
    )
    
    save_location = '/Volumes/JCHD/Microplastics_Paper/Data/Eulerian_3hr/Eulerian'
    save_file_type_npy = '.npy'
    save_file_type_nc = '.nc'
        
    save_path_npy = save_location + str(stamp).zfill(6) + save_file_type_npy
    save_path_nc = save_location + str(stamp).zfill(6) + save_file_type_nc
    
    np.save(save_path_npy, data)
    U_ek.to_netcdf(save_path_nc)
    
    del data
    del output
    del U_ek
    del Us_u_array
    del Us_file
    

37984


0
7
42
56
14
49
35
28
21
63
70
77




1


Process ForkPoolWorker-1:
Process ForkPoolWorker-8:
Process ForkPoolWorker-9:
Process ForkPoolWorker-4:
Process ForkPoolWorker-6:
Process ForkPoolWorker-10:
Process ForkPoolWorker-7:
Process ForkPoolWorker-12:
Process ForkPoolWorker-2:
Process ForkPoolWorker-11:
Process ForkPoolWorker-5:
Process ForkPoolWorker-3:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):


KeyboardInterrupt: 

...


106



99



162



134



92
